In [7]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from numpy import sort
from sklearn.feature_selection import SelectFromModel

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from itertools import combinations

from bs4 import BeautifulSoup as soup
import requests

In [19]:
df = pd.read_csv("Dbacks_Train.csv")
#df = df.drop(["year","name","college","position"],1)
df.head()
# 721

,year,name,college,position,height_in,weight_lbs,40yards,benchpress,vert_leap_in,broad_jump_in,shuttle,3cone,solotackles,sacks,totaltackles,hasawards,allamerican,heisman,drafted
0,1997,ink aleaga,Washington,ILB,73.0,227,4.87,NaN,31.0,107.0,4.33,7.60,NaN,NaN,NaN,0.0,0.0,0.0,N
1,1997,richard alvarado,Jackson State (MS),OLB,74.6,239,4.92,18.0,33.5,115.0,4.28,7.48,NaN,NaN,NaN,0.0,0.0,0.0,N
2,1997,derrick barnes,Oregon,OLB,72.9,261,4.92,15.0,33.0,109.0,4.42,7.88,NaN,NaN,NaN,0.0,0.0,0.0,N
3,1997,james bates,Florida,ILB,73.0,230,4.92,NaN,32.5,115.0,4.29,7.67,NaN,NaN,NaN,0.0,0.0,0.0,N
4,1997,dexter coakley,Appalachian State (NC),OLB,69.6,215,4.52,21.0,38.0,125.0,4.21,7.48,NaN,NaN,NaN,0.0,0.0,0.0,Y


In [18]:
from splinter import Browser
from sys import platform
import os
import csv
import re

executable_path = None

if platform == "linux" or platform == "linux2":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-linux")}
elif platform == "darwin":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-mac")}
elif platform == "win32":
    executable_path = {'executable_path': os.path.join("..","..","binaries", "chromedriver-windows.exe")}

browser = Browser('chrome', **executable_path, headless=False)

In [99]:
for index, row in df.iterrows():
    if (df.loc[index,"allamerican"] != 1.0):
        if (df.loc[index,"allamerican"] != 0.0):
            x = row["name"]
            browser.visit(f"https://www.sports-reference.com/cfb/search/search.fcgi?hint=&search={x}")
            if(len(browser.find_by_css("h1[itemprop='name']")) == 1):
                if (len(browser.find_by_css("li.bling_all_star")) > 0):
                    df.loc[index,"allamerican"] = 1
                else:
                    df.loc[index,"allamerican"] = 0
                if (len(browser.find_by_css("li.bling_hof")) > 0):
                    df.loc[index,"heisman"] = 1
                else:
                    df.loc[index,"heisman"] = 0
                if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                    df.loc[index,"hasawards"] = 1
                else:
                    df.loc[index,"hasawards"] = 0
            elif (len(browser.find_by_css("div.search-item")) == 0):
                df.loc[index,"allamerican"] = 0
                df.loc[index,"heisman"] = 0
                df.loc[index,"hasawards"] = 0
            else:
                listed = browser.find_by_css("div.search-item")

                for x in listed:
                    if (re.sub(r'\(.*\)', "", row["college"]).strip() in x.find_by_css("div.search-item-league").text):
                        x.find_by_css("div.search-item-name a").click()
                        if (len(browser.find_by_css("h1[itemprop='name']")) == 1):
                            if (len(browser.find_by_css("li.bling_all_star")) > 0):
                                df.loc[index,"allamerican"] = 1
                            else:
                                df.loc[index,"allamerican"] = 0
                            if (len(browser.find_by_css("li.bling_hof")) > 0):
                                df.loc[index,"heisman"] = 1
                            else:
                                df.loc[index,"heisman"] = 0
                            if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                                df.loc[index,"hasawards"] = 1
                            else:
                                df.loc[index,"hasawards"] = 0
                        else:
                            df.loc[index,"allamerican"] = 0
                            df.loc[index,"heisman"] = 0
                            df.loc[index,"hasawards"] = 0
                            break
                    else:
                        df.loc[index,"allamerican"] = 0
                        df.loc[index,"heisman"] = 0
                        df.loc[index,"hasawards"] = 0

In [101]:
df.to_csv("Dbacks_train.csv", index=False)